In [27]:
import pandas as pd
import spacy

# Загрузка данных из Excel
df = pd.read_excel("ner_data_train.xlsx")

In [28]:
# Описание тегов
tag_descriptions = {
    "O": "Неопределенный",
    "B-Дата": "Дата",
    "B-бренд": "Бренд",
    "B-вид спорта": "Вид спорта",
    "B-видеоигра": "Видеоигра",
    "B-команда": "Команда",
    "B-лига": "Лига",
    "B-локация": "Локация",
    "B-модель": "Модель",
    "B-название проекта": "Название проекта",
    "B-организация": "Организация",
    "B-персона": "Персона",
    "B-сезон": "Сезон",
    "B-серия": "Серия",
    "I-Дата": "Дата (продолжение)",
    "I-бренд": "Бренд (продолжение)",
    "I-вид спорта": "Вид спорта (продолжение)",
    "I-видеоигра": "Видеоигра (продолжение)",
    "I-команда": "Команда (продолжение)",
    "I-лига": "Лига (продолжение)",
    "I-локация": "Локация (продолжение)",
    "I-модель": "Модель (продолжение)",
    "I-название проекта": "Название проекта (продолжение)",
    "I-организация": "Организация (продолжение)",
    "I-персона": "Персона (продолжение)",
    "I-сезон": "Сезон (продолжение)",
    "I-серия": "Серия (продолжение)",
}

In [29]:

# Загрузка spaCy модели для русского языка
nlp = spacy.load("ru_core_news_lg")

# Функция для извлечения тегов
def extract_tags(text):
    # Проверяем, является ли значение float
    if isinstance(text, float):
        return []
    
    doc = nlp(text)
    tags = [
        {
            "label": tag_descriptions[ent.label_],
            "segment": ent.text
        }
        for ent in doc.ents
        if ent.label_ in tag_descriptions
    ]
    return tags

# Применение функции к данным и создание новой колонки с тегами
df["tags"] = df["video_info"].apply(extract_tags)

# Сохранение результата в новый Excel файл
df.to_excel("ner_data_OU.xlsx", index=True)

# Вывод обработанных данных
print(df)


                                             video_info  \
0     <НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...   
1     <НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 V...   
2     <НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Челов...   
3     <НАЗВАНИЕ:> Довоенная немецкая кирха в Калинин...   
4     <НАЗВАНИЕ:> "Спартаку" помогли судьи? Локомоти...   
...                                                 ...   
6417  <НАЗВАНИЕ:> ЗАПРЕЩЕННЫЙ УДАР! Полный бой Вадим...   
6418  <НАЗВАНИЕ:> Как создаются медали: оптические и...   
6419  <НАЗВАНИЕ:> Ремонт ломовоза камаз, монтаж гидр...   
6420  <НАЗВАНИЕ:> Голодные игры: Баллада о змеях и п...   
6421  <НАЗВАНИЕ:> Экскаватор на Дровах! Часть 15. За...   

                                               entities tags  info  
0     {"label":"локация"\,"offset":26\,"length":6\,"...   []   NaN  
1     {"label":"организация"\,"offset":196\,"length"...   []   NaN  
2     {"label":"название проекта"\,"offset":12\,"len...   []   NaN  
3     {"label":